## Data cleaning and preprocessing

To begin with the analysis, we should firstly clean and prepare our data. This notebook file completes the following steps:
* Separating numerical and categorical data to process more efficiently.
* Renaming features with very long names to make our life easier.
* Encoding our target (dependent) variable.
* Imputing missing values with the most frequent value of the column.
* Dropping redundant and highly correlated numerical features.
* Dropping some of the categorical features that do not contain valuable information for the analysis.
* Encoding categorical features using One-Hot encoding for non-ordinal features, and assigning logical scales to ordinal features.

In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore") # warnings were turned off at the end

In [2]:
data = pd.read_csv('../data/data.csv', encoding='latin1', na_values = "No Info").fillna(np.nan)
obj_cols = data.select_dtypes(include = ["object"]).columns
data[obj_cols] = data[obj_cols].apply(lambda x: x.astype(str).str.lower())
data[obj_cols] = data[obj_cols].replace("nan", np.nan)
data.head()

,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,...,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,company1,success,NaN,NaN,-1.0,video distribution,NaN,operation,kpcb holdings|draper fisher jurvetson (dfj)|kl...,3.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,company2,success,2011.0,3.0,125.0,NaN,market research|marketing|crowdfunding,"marketing, sales",NaN,NaN,...,8.823529,21.764706,10.882353,2.941176,0.000000,0.0,0.0,0.0,0.0,8.0
2,company3,success,2011.0,3.0,455.0,event data analytics api,analytics|cloud computing|software development,operations,techstars|streamlined ventures|amplify partner...,14.0,...,3.846154,17.094017,9.401709,0.000000,2.777778,0.0,0.0,0.0,0.0,9.0
3,company4,success,2009.0,5.0,-99.0,the most advanced analytics for mobile,mobile|analytics,marketing & sales,michael birch|max levchin|sequoia capital|keit...,45.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,5.0
4,company5,success,2010.0,4.0,496.0,the location-based marketing platform,analytics|marketing|enterprise software,marketing & sales,dfj frontier|draper nexus ventures|gil elbaz|a...,39.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,6.0


In [3]:
data.drop("Company_Name", axis = 1, inplace = True)

In [4]:
data.shape

(472, 115)

In [5]:
data.rename({
    "Dependent-Company Status": "target",
    "Avg time to investment - average across all rounds, measured from previous investment": "avg_inv",
    "Survival through recession, based on existence of the company through recession times": "rec_survival",
    "Time to maturity of technology (in years)": "tech_maturity_years",
    "Last round of funding received (in milionUSD)": "last_funding_mil",
    'Age of company in years': "age",
    'Short Description of company profile': "short_desc",
    'Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive': "capital_intensive",
    'Owns data or not? (monetization of data) e.g. Factual': "owns_data",
    'Is the company an aggregator/market place? e.g. Bluekai': "aggregator_market",
    'Online or offline venture - physical location based business or online venture?': "online_offline",
    "Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?": "top_forum_talk",
    'Average Years of experience for founder and co founder': "founders_avg_exp",
    'Presence of a top angel or venture fund in previous round of investment': "prev_round_angel_venture",
    'Have been part of startups in the past?': "startup_presence",
    'Have been part of successful startups in the past?': "success_startup_presence",
    'Was he or she partner in Big 5 consulting?': "big_5_consulting",
    'Technical proficiencies to analyse and interpret unstructured data': "analyse_unstructured_data",
    'Number of Recognitions for Founders and Co-founders': "founders_recognitions"
}, axis = "columns", inplace = True)


data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(" ", "_")

## Processing

Lets encode target variable.

In [6]:
def label_encoder(columns, df, show_percents = False):
    for column in columns:
        encoder = LabelEncoder()
        df[column] = encoder.fit_transform(df[column])
    
    if show_percents == True:
        for label in list(df[column].unique()):
            count = df[column].value_counts()[label]
            percent = str(np.round(df[column].value_counts()[label]/df.shape[0] * 100)) + " %"
            print(f"Class {label}:" , count, "|", percent)

In [7]:
label_encoder(["target"], data, show_percents = True)

Class 1: 305 | 65.0 %
Class 0: 167 | 35.0 %


Here we can see that the majority of startups succeed, which may potentially lead to the class imbalance problem later.

Let us examine the numerical features and understand their behaviour.

In [8]:
num_data = data.select_dtypes(include = ["float64", "int64"])

num_corrs = data[list(num_data.columns)].corr().abs().unstack().sort_values(kind = "quicksort").reset_index()
num_corrs = num_corrs[num_corrs['level_0'] != num_corrs['level_1']].drop_duplicates(0).reset_index(drop = True)
num_corrs.tail(10)

,level_0,level_1,0
850,employees_per_year_of_company_existence,last_funding_amount,0.417713
851,employee_count,time_to_1st_investment_(in_months),0.430506
852,employee_count,team_size_senior_leadership,0.497193
853,experience_in_fortune_500_organizations,experience_in_fortune_100_organizations,0.512101
854,age,time_to_1st_investment_(in_months),0.561613
855,time_to_1st_investment_(in_months),year_of_founding,0.561613
856,time_to_1st_investment_(in_months),avg_inv,0.692256
857,experience_in_fortune_500_organizations,experience_in_fortune_1000_organizations,0.694509
858,team_size_all_employees,employees_per_year_of_company_existence,0.956601
859,age,year_of_founding,1.000000


Above we can see top-10 correlated features. The features with high correlation are:
* `age` and `year_of_founding`. They are opposite of one another.
* `team_size_all_employees` and `employees_per_year_of_company_existence`. The number of employees did not change much from the beginning.
* `experience_in_fortune_100_organizations`, `experience_in_fortune_500_organizations` and `experience_in_fortune_500_organizations`.
* `time_to_1st_investment_(in_months)` and `avg_inv`. Companies may receive only one investment, which may lead to the repeating observations between these columns.

In [9]:
drop_cols = ["year_of_founding",
             "employees_per_year_of_company_existence", 
             "experience_in_fortune_1000_organizations", 
             "time_to_1st_investment_(in_months)"]

data.drop(drop_cols, axis = 1, inplace = True)
num_data.drop(drop_cols, axis = 1, inplace = True)

### Missing Values

In [10]:
def impute_mode(df, exclude):
    for column in df.select_dtypes(exclude = exclude):
        df[column].fillna(df[column].mode()[0], inplace = True)
    return df

In [11]:
data = impute_mode(data, exclude = ["object"])
print("Null count (numeric variables):", data.select_dtypes(exclude=['object']).isnull().sum().sum())

Null count (numeric variables): 0


### Categorical features

In [12]:
cat_data = data.select_dtypes(include = ['object'])
cat_data.head()

,short_desc,industry_of_company,focus_functions_of_company,investors,has_the_team_size_grown,est._founding_date,last_funding_date,country_of_company,continent_of_company,prev_round_angel_venture,...,legal_risk_and_intellectual_property,client_reputation,analyse_unstructured_data,solutions_offered,invested_through_global_incubation_competitions?,disruptiveness_of_technology,last_funding_mil,rec_survival,gartner_hype_cycle_stage,tech_maturity_years
0,video distribution,NaN,operation,kpcb holdings|draper fisher jurvetson (dfj)|kl...,no,NaN,5/26/2013,united states,north america,yes,...,no,NaN,no,yes,no,low,0.45,NaN,NaN,NaN
1,NaN,market research|marketing|crowdfunding,"marketing, sales",NaN,no,NaN,NaN,united states,north america,no,...,yes,medium,yes,yes,no,medium,5,not applicable,trough,2 to 5
2,event data analytics api,analytics|cloud computing|software development,operations,techstars|streamlined ventures|amplify partner...,no,12/1/2011,10/23/2013,united states,north america,no,...,no,low,yes,yes,yes,medium,2.35,not applicable,trough,2 to 5
3,the most advanced analytics for mobile,mobile|analytics,marketing & sales,michael birch|max levchin|sequoia capital|keit...,no,6/20/2009,5/10/2012,united states,north america,yes,...,no,low,no,yes,yes,medium,10.25,not applicable,trough,2 to 5
4,the location-based marketing platform,analytics|marketing|enterprise software,marketing & sales,dfj frontier|draper nexus ventures|gil elbaz|a...,no,4/1/2010,12/11/2013,united states,north america,no,...,no,low,yes,yes,no,medium,5.5,not applicable,NaN,NaN


In [13]:
redundants = ["rec_survival", "est._founding_date", "last_funding_date", "last_funding_mil"]

cat_data.drop(redundants, axis = 1, inplace = True)
data.drop(redundants, axis = 1, inplace = True)

### Identifying, imputing and labeling columns with yes, no observations

In [14]:
def yes_no_cols(df):
    yes_no = []
    for column in df.columns:
        if any(cat_data[column] == "yes"):
            yes_no.append(str(column))
    return yes_no

yes_no = yes_no_cols(cat_data)
print("Number of columns with yes, no values:", len(yes_no))

Number of columns with yes, no values: 35


In [15]:
data[yes_no] = impute_mode(data[yes_no], exclude = ["float64", "int64"])
cat_data[yes_no] = impute_mode(cat_data[yes_no], exclude = ["float64", "int64"])

label_encoder(yes_no, data)
label_encoder(yes_no, cat_data)

print("Null count yes, no:", data[yes_no].select_dtypes(include=['object']).isnull().sum().sum())

Null count yes, no: 0.0


### Analysing remaining features of the type object

In [16]:
remain_cat = cat_data.select_dtypes(include = "object")

Several columns containing ordinal values.

In [17]:
def ordinal_cols(df):
    ordinals = []
    for column in df.columns:
        if any((df[column] == "medium") | (df[column] == "bad") | (df[column] == "few")):
            ordinals.append(str(column))
    return ordinals

In [18]:
ord_cols = ordinal_cols(remain_cat)
data[ord_cols] = impute_mode(data[ord_cols], exclude = ["float64", "int64"])
remain_cat[ord_cols] = impute_mode(remain_cat[ord_cols], exclude = ["float64", "int64"])

for column in ord_cols:
    print(remain_cat[column].unique())

['nothing' 'medium' 'low' 'high']
['small' 'large' 'medium']
['none' 'few' 'many']
['high' 'low' 'medium' 'none']
['high' 'medium' 'low']
['low' 'high' 'medium']
['medium' 'high' 'low' 'none']
['none' 'medium' 'low' 'high']
['none' 'few' 'many']
['low' 'high' 'medium']
['low' 'medium' 'high']
['high' 'low' 'medium']
['bad' 'good' 'average' 'very good']
['low' 'medium' 'high']
['low' 'medium' 'high']


In [19]:
def scale_mapping(df, ordinals):
    scale_map = {
        "nothing": 0,
        "none": 0,
        "low": 1,
        "bad": 1,
        "small": 1,
        "few": 1,
        "medium": 2,
        "many": 2,
        "good": 2,
        "large": 3,
        "high": 3,
        "average": 3,
        "very good": 4
    }
    
    for column in ordinals:
        df[column] = df[column].replace(scale_map)
    
    return df

In [20]:
remain_cat[ord_cols] = scale_mapping(remain_cat[ord_cols], ord_cols)
data[ord_cols] = scale_mapping(data[ord_cols], ord_cols)

data.head()

,target,age,internet_activity_score,short_desc,industry_of_company,focus_functions_of_company,investors,employee_count,employees_count_mom_change,has_the_team_size_grown,...,percent_skill_data_science,percent_skill_business_strategy,percent_skill_product_management,percent_skill_sales,percent_skill_domain,percent_skill_law,percent_skill_consulting,percent_skill_finance,percent_skill_investment,renown_score
0,1,4.0,-1.0,video distribution,NaN,operation,kpcb holdings|draper fisher jurvetson (dfj)|kl...,3.0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,3.0,125.0,NaN,market research|marketing|crowdfunding,"marketing, sales",NaN,0.0,0.0,0,...,8.823529,21.764706,10.882353,2.941176,0.000000,0.0,0.0,0.0,0.0,8.0
2,1,3.0,455.0,event data analytics api,analytics|cloud computing|software development,operations,techstars|streamlined ventures|amplify partner...,14.0,0.0,0,...,3.846154,17.094017,9.401709,0.000000,2.777778,0.0,0.0,0.0,0.0,9.0
3,1,5.0,-99.0,the most advanced analytics for mobile,mobile|analytics,marketing & sales,michael birch|max levchin|sequoia capital|keit...,45.0,10.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,5.0
4,1,4.0,496.0,the location-based marketing platform,analytics|marketing|enterprise software,marketing & sales,dfj frontier|draper nexus ventures|gil elbaz|a...,39.0,3.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,6.0


In [21]:
remain_cat.drop(ord_cols, axis = 1, inplace = True)

In [22]:
remain_cat.columns

Index(['short_desc', 'industry_of_company', 'focus_functions_of_company',
       'investors', 'country_of_company', 'continent_of_company',
       'product_or_service_company?', 'focus_on_private_or_public_data?',
       'focus_on_structured_or_unstructured_data',
       'cloud_or_platform_based_serive/product?', 'local_or_global_player',
       'linear_or_non-linear_business_model', 'online_offline',
       'b2c_or_b2b_venture?', 'highest_education',
       'specialization_of_highest_education',
       'degree_from_a_tier_1_or_tier_2_university?',
       'gartner_hype_cycle_stage', 'tech_maturity_years'],
      dtype='object')

In [23]:
rem_cols = ["short_desc",
            "industry_of_company",
            "focus_functions_of_company",
            "investors",
            "country_of_company", 
            "specialization_of_highest_education"]

remain_cat.drop(rem_cols, axis = 1, inplace = True)
data.drop(rem_cols, axis = 1, inplace = True)

In [24]:
remain_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 13 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   continent_of_company                        401 non-null    object
 1   product_or_service_company?                 462 non-null    object
 2   focus_on_private_or_public_data?            463 non-null    object
 3   focus_on_structured_or_unstructured_data    463 non-null    object
 4   cloud_or_platform_based_serive/product?     460 non-null    object
 5   local_or_global_player                      448 non-null    object
 6   linear_or_non-linear_business_model         454 non-null    object
 7   online_offline                              466 non-null    object
 8   b2c_or_b2b_venture?                         469 non-null    object
 9   highest_education                           369 non-null    object
 10  degree_from_a_tier_1_or_ti

In [25]:
remain_cat["degree_from_a_tier_1_or_tier_2_university?"].value_counts()

none      144
tier_1    139
tier_2     58
both       43
Name: degree_from_a_tier_1_or_tier_2_university?, dtype: int64

In [26]:
remain_cat["tech_maturity_years"].value_counts()

2 to 5     180
0 to 2      77
5 to 10     42
0 to 5       1
Name: tech_maturity_years, dtype: int64

In [27]:
new_ord_cols = ["degree_from_a_tier_1_or_tier_2_university?", "tech_maturity_years"]
data[new_ord_cols] = impute_mode(data[new_ord_cols], exclude = ["float64", "int64"])
remain_cat[new_ord_cols] = impute_mode(remain_cat[new_ord_cols], exclude = ["float64", "int64"])

new_scales = {
    "none": 1,
    "tier_2": 2,
    "5 to 10": 1,
    "tier_1": 3,
    "2 to 5": 2,
    "0 to 2": 3,
    "0 to 5": 3,
    "both": 4
}

for column in new_ord_cols:
    remain_cat[column] = remain_cat[column].replace(new_scales)
    data[column] = data[column].replace(new_scales)

In [28]:
remain_cat.drop(new_ord_cols, axis = 1, inplace = True)

In [29]:
one_hots = remain_cat.columns
data[one_hots] = impute_mode(data[one_hots], exclude = ["float64", "int64"])
remain_cat = impute_mode(remain_cat, exclude = ["float64", "int64"])

data = pd.get_dummies(data, columns = one_hots)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Columns: 128 entries, target to gartner_hype_cycle_stage_trough
dtypes: float64(34), int64(56), uint8(38)
memory usage: 349.5 KB


In [30]:
data.to_csv("../data/clean_data.csv")